In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
#import libraries and some constants

import os
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize, LogNorm
import pandas as pd
import uproot as ur
import pickle
import atlas_mpl_style as ampl
ampl.use_atlas_style()

path_prefix = '/AL/Phd/maxml/caloml-atlas/'
plotpath = path_prefix+'classifier/Plots/'
modelpath = path_prefix+'classifier/Models/'

# import our resolution utilities
import sys
sys.path.append(path_prefix)
from  util import resolution_util as ru
from  util import plot_util as pu
from  util import ml_util as mu

inputpath = path_prefix+'inputs/'
rootfiles = ["pi0", "piplus", "piminus"]

trees, pdata = mu.setupPionData(inputpath, rootfiles)

Using TensorFlow backend.


In [3]:
np0 = len(pdata['pi0'])
npp = len(pdata['piplus'])
npm = len(pdata['piminus'])

print("Number of pi0 events: {}".format(np0))
print("Number of pi+ events: {}".format(npp))
print("Number of pi- events: {}".format(npm))
print("Total: {}".format(np0+npp+npm))

pcells = {
    ifile : {
        layer : mu.setupCells(itree, layer, flatten = False)
        for layer in mu.cell_meta
    }
    for ifile, itree in trees.items()
}

Number of pi0 events: 263891
Number of pi+ events: 435967
Number of pi- events: 434627
Total: 1134485


In [4]:
import numpy
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Convolution2D
from tensorflow.keras.layers import MaxPool2D
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import concatenate
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Model
from tensorflow import keras as keras

import tensorflow as tf

gpu_list = ["/gpu:0"]
strategy = tf.distribute.MirroredStrategy(devices=gpu_list)
# strategy = tf.distribute.MirroredStrategy()
ngpu = strategy.num_replicas_in_sync
print ('Number of devices: {}'.format(ngpu))

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
Number of devices: 1


In [5]:
training_classes = ['pi0','piplus']
# create train/validation/test subsets containing 70%/10%/20%
# of events from each type of pion event
pdata_merged, pcells_merged, plabels = mu.createTrainingDatasets(training_classes, pdata, pcells)

pcells_merged_reshaped = mu.reshapeSeparateCNN(pcells_merged)

In [6]:
filters = {
    'EMB1': (1,1), 
    'EMB2': (1,1), 
    'EMB3': (1,1), 
    'TileBar0': (1,1), 
    'TileBar1': (1,1), 
    'TileBar2': (1,1)
}
filters2 = {
    'EMB1': (4,8), 
    'EMB2': (8,8), 
    'EMB3': (4,2), 
    'TileBar0': (2,2), 
    'TileBar1': (2,2), 
    'TileBar2': (1,1)
}
pools2 = {
    'EMB1': (2,2), 
    'EMB2': (2,2), 
    'EMB3': (1,1), 
    'TileBar0': (1,1), 
    'TileBar1': (1,1), 
    'TileBar2': (1,1)
}

In [7]:
def cnn_model_layers(layer):
    print(layer)
    # create model
    with strategy.scope():
        model = Sequential()
        model.add(Convolution2D(32, filters[layer], input_shape=(1,mu.cell_meta[layer]['len_eta'],mu.cell_meta[layer]['len_phi']), activation='relu', data_format = 'channels_first'))
        model.add(MaxPool2D(pool_size=(2, 2)))
        model.add(Convolution2D(16, pools2[layer], activation='relu'))
        model.add(MaxPool2D(pool_size=pools2[layer]))
        model.add(Flatten())
        model.add(Dense(128, activation='relu'))
        model.add(Dense(64, activation='relu'))
        model.add(Dense(2, kernel_initializer='normal', activation='softmax'))
        # compile model
        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])
        return model

In [8]:
models = {
    layer: cnn_model_layers(layer)
    for layer in mu.cell_meta
}

EMB1
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
EMB2
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
EMB3
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
TileBar0
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0'

In [ ]:
history = {
    layer: models[layer].fit(pcells_merged_reshaped[layer][pdata_merged.train], 
                                plabels[pdata_merged.train], 
                                validation_data=(pcells_merged_reshaped[layer][pdata_merged.val], plabels[pdata_merged.val]),                                   epochs=100, batch_size=200*ngpu, verbose=2)
    for layer in mu.cell_meta
}

Train on 489899 samples, validate on 69988 samples
Epoch 1/100
489899/489899 - 31s - loss: 0.4050 - acc: 0.8187 - val_loss: 0.3827 - val_acc: 0.8354
Epoch 2/100
489899/489899 - 23s - loss: 0.3403 - acc: 0.8603 - val_loss: 0.2971 - val_acc: 0.8847
Epoch 3/100
489899/489899 - 23s - loss: 0.2909 - acc: 0.8861 - val_loss: 0.2845 - val_acc: 0.8901
Epoch 4/100
489899/489899 - 23s - loss: 0.2779 - acc: 0.8913 - val_loss: 0.2708 - val_acc: 0.8966
Epoch 5/100
489899/489899 - 23s - loss: 0.2713 - acc: 0.8940 - val_loss: 0.2666 - val_acc: 0.8965
Epoch 6/100
489899/489899 - 23s - loss: 0.2661 - acc: 0.8964 - val_loss: 0.2679 - val_acc: 0.8976
Epoch 7/100
489899/489899 - 23s - loss: 0.2612 - acc: 0.8984 - val_loss: 0.2629 - val_acc: 0.9004
Epoch 8/100
489899/489899 - 23s - loss: 0.2579 - acc: 0.8998 - val_loss: 0.2583 - val_acc: 0.9006
Epoch 9/100
489899/489899 - 23s - loss: 0.2563 - acc: 0.9005 - val_loss: 0.2599 - val_acc: 0.9005
Epoch 10/100
489899/489899 - 23s - loss: 0.2544 - acc: 0.9010 - val

In [ ]:
# save networks

for layer, model in models.items():
    model.save(modelpath+"model_cnn_1x1_"+layer+".h5")
    with open(modelpath + "model_cnn_1x1_"+layer+".history",'wb') as model_history_file:
        pickle.dump(history[layer].history, model_history_file)